In [10]:
import requests
from bs4 import BeautifulSoup
import folium

url = 'https://vancouver.craigslist.org/search/bia'
bicycleLinks = []
bicycleSpatialData = []

# Send a GET request to the URL
response = requests.get(url)

# Create a BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

# Find all the links on the page
links = soup.find_all('a', href=lambda href: href and '/bik/' in href)
print(links)
# Iterate over each link
linkcount = 0
for link in links:
    if linkcount > 100:
        break
    linkcount += 1
    href = link['href']
    bicycleLinks.append(href)

    # Visit each link to collect spatial data
    link_response = requests.get(href)
    link_soup = BeautifulSoup(link_response.text, 'html.parser')

    # Find the map div and extract latitude and longitude
    map_div = link_soup.find('div', id='map')
    latitude = float(map_div['data-latitude'])
    longitude = float(map_div['data-longitude'])

    bicycleSpatialData.append([latitude, longitude])

# Create a map of Greater Vancouver
vancouver_map = folium.Map(location=[49.2827, -123.1207], zoom_start=10)

# Create a feature group for the bicycle spatial data layer
bicycle_layer = folium.FeatureGroup(name='Bicycle Spatial Data')

# Add markers for each latitude and longitude point in bicycleSpatialData
for spatial_data in bicycleSpatialData:
    latitude, longitude = spatial_data
    marker = folium.Marker(location=[latitude, longitude])
    bicycle_layer.add_child(marker)

# Add the bicycle spatial data layer to the map
vancouver_map.add_child(bicycle_layer)

# Add a layer control to toggle the bicycle spatial data layer
folium.LayerControl().add_to(vancouver_map)

# Save the map to an HTML file
vancouver_map.save('vancouver_map.html')

print("Map created successfully!")


[]
Map created successfully!
